In [2]:
import pandas as pd
import numpy as np
import sqlite3 as db

In [3]:
%pwd

'/home/sarah.souchon@Digital-Grenoble.local/Sarah/BDD'

In [4]:
#Import des fichiers --> df
routes_df=pd.read_csv('./SEM_GTMS_TAG/routes.txt')
stops_df=pd.read_csv('./SEM_GTMS_TAG/stops.txt')
stop_times_df=pd.read_csv('./SEM_GTMS_TAG/stop_times.txt')
agency_df=pd.read_csv('./SEM_GTMS_TAG/agency.txt')
trips_df=pd.read_csv('./SEM_GTMS_TAG/trips.txt')

In [5]:
print(routes_df.shape)
print(stops_df.shape)
print(stop_times_df.shape)
print(agency_df.shape)
print(trips_df.shape)

(55, 7)
(2501, 9)
(447215, 6)
(1, 6)
(19669, 8)


In [6]:
#Transformation des fichiers en dictionnaires
routes_dic=routes_df.to_dict(orient='dict')
stops_dic=stops_df.to_dict(orient='dict')
stop_times_dic=stop_times_df.to_dict(orient='dict')
agency_dic=agency_df.to_dict(orient='dict')
trips_dic=trips_df.to_dict(orient='dict')

In [7]:
routes_dic

{'agency_id': {0: 'SEM',
  1: 'SEM',
  2: 'SEM',
  3: 'SEM',
  4: 'SEM',
  5: 'SEM',
  6: 'SEM',
  7: 'SEM',
  8: 'SEM',
  9: 'SEM',
  10: 'SEM',
  11: 'SEM',
  12: 'SEM',
  13: 'SEM',
  14: 'SEM',
  15: 'SEM',
  16: 'SEM',
  17: 'SEM',
  18: 'SEM',
  19: 'SEM',
  20: 'SEM',
  21: 'SEM',
  22: 'SEM',
  23: 'SEM',
  24: 'SEM',
  25: 'SEM',
  26: 'SEM',
  27: 'SEM',
  28: 'SEM',
  29: 'SEM',
  30: 'SEM',
  31: 'SEM',
  32: 'SEM',
  33: 'SEM',
  34: 'SEM',
  35: 'SEM',
  36: 'SEM',
  37: 'SEM',
  38: 'SEM',
  39: 'SEM',
  40: 'SEM',
  41: 'SEM',
  42: 'SEM',
  43: 'SEM',
  44: 'SEM',
  45: 'SEM',
  46: 'SEM',
  47: 'SEM',
  48: 'SEM',
  49: 'SEM',
  50: 'SEM',
  51: 'SEM',
  52: 'SEM',
  53: 'SEM',
  54: 'SEM'},
 'route_id': {0: '1',
  1: '2',
  2: '3',
  3: '4',
  4: '5',
  5: '6',
  6: '7',
  7: '12',
  8: '13',
  9: '14',
  10: '15',
  11: '16',
  12: '19',
  13: '20',
  14: '21',
  15: '22',
  16: '23',
  17: '25',
  18: '26',
  19: '40',
  20: '41',
  21: '42',
  22: '44',
  23: '45'

In [8]:
# Fonction transformation données mais pas création de fichier sql
#= générer une requête SQL INSERT en se basant sur le nom de la table et un dictionnaire 
#de valeurs fournis en entrée
def gen_insert_query(table_name:str, a_dict:dict) -> str:
    #table_name= stops_insert.sql par exple
    #df --> attention à gérer les NaN
    # https://sql.sh/cours/insert-into
    #INSERT INTO table_name (columns name)  VALUES (a_dict.values) 

    insert_query_tpl = "INSERT INTO {} {} VALUES {} "
    #séparer les clés et les valeurs du dictionnaire
    keys, values = zip(*a_dict.items())
    
    compiled_query = insert_query_tpl.format(table_name, keys, values)
    
    return compiled_query

In [9]:
gen_insert_query(routes_df,routes_dic)

'INSERT INTO    agency_id route_id route_short_name  \\\n0        SEM        1               C1   \n1        SEM        2               C2   \n2        SEM        3               C3   \n3        SEM        4               C4   \n4        SEM        5               C5   \n5        SEM        6               C6   \n6        SEM        7               C7   \n7        SEM       12               12   \n8        SEM       13               13   \n9        SEM       14               14   \n10       SEM       15               15   \n11       SEM       16               16   \n12       SEM       19               19   \n13       SEM       20               20   \n14       SEM       21               21   \n15       SEM       22               22   \n16       SEM       23               23   \n17       SEM       25               25   \n18       SEM       26               26   \n19       SEM       40               40   \n20       SEM       41               41   \n21       SEM       42               42  

In [10]:
#Fonction pour mettre en forme .txt puis création fichier sql
#est destinée à générer un fichier contenant des instructions SQL pour insérer les données 
#d'un DataFrame dans une table
def gen_insert_file(filename,tablename,df):
    with open(filename,'w') as file:
        file.write("BEGIN;") #BEGIN et COMMIT sont à ajouter pour optimiser le process/plus rapide
        for i in range(len(df)):
            line=df.iloc[i].dropna()
            query= "INSERT INTO {} {} VALUES {};\n ".format(tablename, tuple(line.index), tuple(line.to_list()))
            file.write(query)
        file.write("COMMIT;")
    return filename
#Pour import dans DB --> terminal : sqlite3 gtfs_tag.db < routes.sql

In [11]:
#INSERER AVEC LA METHODE BULK INSERT
print(gen_insert_file("./routes.sql","routes", routes_df))
gen_insert_file("./routes.sql","routes", routes_df)
gen_insert_file("./stops.sql","stops", stops_df)
gen_insert_file("./stop_times.sql","stop_times", stop_times_df)
gen_insert_file("./agency.sql","agency", agency_df)
gen_insert_file("./trips.sql","trips", trips_df)
                
            

./routes.sql


'./trips.sql'

DANS LE TERMINAL
sqlite3 gtfs_tag.db < routes.sql
sqlite3 gtfs_tag.db < routes.sql

DANS DB BROWSER
onglet executable sql

In [20]:
#INSERER AVEC LA METHODE PANDAS
import pandas
import sqlite3 as db
#crée une connexion à la base de données SQLite en utilisant le nom de fichier "DB.db
DB = db.connect('gtfs_tag.db')
#lire un fichier CSV dans un objet Pandas DataFrame
csv1 = pandas.read_csv('./SEM_GTMS_TAG/agency.txt')
csv2 = pandas.read_csv('./SEM_GTMS_TAG/stops.txt')
#insérer les données du DataFrame Pandas dans une table SQLite appelée "Orders"
csv1.to_sql(name='Orders', con=DB, if_exists='replace')
csv2.to_sql(name='Orders', con=DB, if_exists='replace')
# #replace is one of three options available for the if_exists parameter
#Fermer la connexion à la BDD
DB.close()

In [21]:
#INSERER AVEC LA METHODE 
import sqlite3
#Création du connecteur
DB=db.connect('gtfs_tag.db')
connecteur=sqlite3.connect('gtfs_tag.db')


routes_df.to_sql(name='routes',con=connecteur, if_exists='replace') #if_exists='append' quand on veut rajouter

#Transformations et chargements des fichiers concernés
stops_df=pd.read_csv('./SEM_GTMS_TAG/stops.txt') #ajouter na_values?
stops_df.to_sql(name='stops',con=connecteur, if_exists='replace')

stop_times_df=pd.read_csv('./SEM_GTMS_TAG/stop_times.txt')
stop_times_df.to_sql(name='stops_times',con=connecteur, if_exists='replace')

agency_df=pd.read_csv('./SEM_GTMS_TAG/agency.txt')
agency_df.to_sql(name='agency',con=connecteur, if_exists='replace')

trips_df=pd.read_csv('./SEM_GTMS_TAG/trips.txt')
trips_df.to_sql(name='trips',con=connecteur, if_exists='replace')

#Fermeture de la connection
DB.close()
                
                


    BEGIN : il marque le début d'une transaction. Toutes les instructions SQL qui suivent après le BEGIN sont considérées comme faisant partie de la transaction. Si une erreur se produit au cours de la transaction, toutes les modifications apportées à la base de données depuis le BEGIN seront annulées.

    COMMIT : il marque la fin d'une transaction. Toutes les instructions SQL qui ont été exécutées depuis le BEGIN seront validées et les modifications apportées à la base de données seront enregistrées de manière permanente.

En résumé, BEGIN et COMMIT permettent de garantir l'intégrité des données en s'assurant que toutes les opérations d'une transaction sont soit toutes exécutées, soit toutes annulées en cas d'erreur.

## Plan A:

- Formaliser le schema.sql à importer dans USer interface for sql

- Utiliser pandas pour créer la DB:

    a - DB = db.connect("name.db")

    b - connecteur=sqlite3.connect("name.db")
    
    c - importer ton fichier txt et le transformer en dataframe --> pd.read_csv("fichier.txt")
    
    d - transformer ton df en fichier sql -->        
        namedf.to_sql(name='nomdelatable',con=connecteur,if_exists='replace ou append')
        # append will allow to keys to be preserved like defined in the schema
        #index=False if you do not want extra-index to be added automatically
    
    e - DB.close()
    
    
## Plan B:

- Formaliser le schema.sql à importer dans USer interface for sql

- Recréer la fonction permettant d'implémenter le sql

- Ecriture dans DB : dans terminal --> sqlite3 fichier.db < fichier.sql 



DEUX METHODES D'INSERTION DE DONNEES
PLAN A : Bulk insert plus rapide et adapté au BDD relationnelle, données volumineuses.
PLAN B :Pandas insert, si environnement est pandas et données structurées, nettoyées et transformées.
Le choix se fait également selon les ressources disponibles dans notre environnement.